# Bi-LSTM Spoofing Detection for Maritime AIS Data

Implementation based on "Vessel Trajectory Route Spoofed Points Detection Using AIS Data: A Bi-LSTM Approach" (Raj & Kumar, 2025)

**Methodology:**
1. AIS Data Loading from Full Dataset (2017-2019)
2. Data Preprocessing & Feature Extraction
3. Spoofed Point Generation (Data Augmentation)
4. Bi-LSTM Model Architecture
5. Training & Evaluation
6. Anomaly Detection & Validation

In [1]:
# Environment setup and imports
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # Fix OpenMP conflict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Deep Learning - PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Visualization
import folium
from folium import plugins
import itertools

print(f"PyTorch version: {torch.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print("All packages loaded successfully")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

PyTorch version: 2.9.1+cu126
GPU Available: True
GPU Device: NVIDIA GeForce RTX 4060 Laptop GPU
Numpy version: 1.26.0
Pandas version: 2.3.3
All packages loaded successfully
Using device: cuda


## 1. Data Loading & Preprocessing

Load AIS data from incident slices created in the incident analysis notebook.

In [2]:
# Configuration
data_root = Path("../dataset/piraeus")
output_root = Path("./models")
output_root.mkdir(parents=True, exist_ok=True)

# AIS dataset configuration (from incident_anomaly_labels.ipynb)
cols_primary = ["timestamp", "vessel_id", "lon", "lat", "speed", "course", "heading"]
cols_alias = ["t", "timestamp", "vessel_id", "lon", "lat", "speed", "course", "heading"]

MONTH_ABBR = {
    1: "jan", 2: "feb", 3: "mar", 4: "apr", 5: "may", 6: "jun",
    7: "jul", 8: "aug", 9: "sep", 10: "oct", 11: "nov", 12: "dec"
}

# Years and months to load for comprehensive training
DATA_PERIODS = [
    (2017, [5, 6, 7, 8, 9, 10, 11, 12]),  # May-Dec 2017
    (2018, list(range(1, 13))),            # Full year 2018
    (2019, list(range(1, 13))),            # Full year 2019
]

print("Configuration loaded")
print(f"Data root: {data_root}")
print(f"Loading periods: 2017-2019 (multiple months)")

Configuration loaded
Data root: ..\dataset\piraeus
Loading periods: 2017-2019 (multiple months)


In [3]:
# Load AIS data from full dataset
def load_month(year, month, root, chunk_size=500_000):
    """Load AIS data for a specific month (from incident_anomaly_labels.ipynb)."""
    folder = Path(root) / f"unipi_ais_dynamic_{year}"
    fname = folder / f"unipi_ais_dynamic_{MONTH_ABBR[month]}{year}.csv"
    if not fname.exists():
        alt = folder / f"unipi_ais_dynamic_{year}_{month:02d}.csv"
        if alt.exists():
            fname = alt
        else:
            raise FileNotFoundError(f"Missing file: {fname} (or {alt})")

    # Discover available columns
    preview = pd.read_csv(fname, nrows=1)
    available = list(preview.columns)
    selected_cols = [c for c in cols_alias if c in available]

    chunks = []
    for chunk in pd.read_csv(fname, usecols=selected_cols, chunksize=chunk_size):
        if "t" in chunk.columns and "timestamp" not in chunk.columns:
            chunk = chunk.rename(columns={"t": "timestamp"})
        chunk["timestamp"] = pd.to_datetime(chunk["timestamp"], unit="ms", utc=True)
        chunk = chunk[[c for c in cols_primary if c in chunk.columns]]
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)


def load_ais_dataset(data_periods, root, sample_months=3):
    """Load multiple months of AIS data for training."""
    all_data = []
    loaded_count = 0

    for year, months in data_periods:
        # Sample subset of months to manage memory
        selected_months = np.random.choice(months, min(sample_months, len(months)), replace=False)
        
        for month in selected_months:
            try:
                print(f"Loading {year}-{month:02d}...")
                df = load_month(year, month, root)
                all_data.append(df)
                loaded_count += 1
                print(f"  Loaded: {len(df):,} records, {df['vessel_id'].nunique()} vessels")
            except FileNotFoundError as e:
                print(f"  Skipped: {e}")
                continue
            except Exception as e:
                print(f"  Error: {e}")
                continue
    
    if not all_data:
        raise FileNotFoundError("No AIS data found. Check data_root path.")
    
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\nTotal months loaded: {loaded_count}")
    return combined_df


# Load data (sample 3 months per year for manageable dataset size)
print("Loading AIS dataset...")
ais_data = load_ais_dataset(DATA_PERIODS, data_root, sample_months=3)

print(f"\nDataset Summary:")
print(f"  Total records: {len(ais_data):,}")
print(f"  Unique vessels: {ais_data['vessel_id'].nunique()}")
print(f"  Time range: {ais_data['timestamp'].min()} to {ais_data['timestamp'].max()}")
print(f"  Columns: {list(ais_data.columns)}")

Loading AIS dataset...
Loading 2017-10...
  Loaded: 4,286,717 records, 816 vessels
Loading 2017-05...
  Loaded: 4,305,035 records, 778 vessels
Loading 2017-07...
  Loaded: 8,340,504 records, 1135 vessels
Loading 2018-04...
  Loaded: 7,608,586 records, 884 vessels
Loading 2018-01...
  Loaded: 5,458,182 records, 659 vessels
Loading 2018-12...
  Loaded: 4,690,581 records, 848 vessels
Loading 2019-10...
  Loaded: 10,944,763 records, 1262 vessels
Loading 2019-02...
  Loaded: 6,930,667 records, 874 vessels
Loading 2019-01...
  Loaded: 4,480,731 records, 782 vessels

Total months loaded: 9

Dataset Summary:
  Total records: 57,045,766
  Unique vessels: 3846
  Time range: 2017-05-09 14:05:26+00:00 to 2019-10-30 16:01:07+00:00
  Columns: ['timestamp', 'vessel_id', 'lon', 'lat', 'speed', 'course', 'heading']


In [4]:
# Data cleaning (Algorithm 1 from paper: domain-specific constraints)
def clean_ais_data(df):
    """Apply domain-specific constraints as per paper."""
    original_len = len(df)
    
    # Rule-based cleaning from paper
    df = df.copy()
    
    # COG should not exceed 360 degrees
    if 'course' in df.columns:
        df = df[df['course'] <= 360]
    
    # SOG should not exceed 100 knots
    if 'speed' in df.columns:
        df = df[df['speed'] <= 100]
    
    # Remove null values in critical fields
    df = df.dropna(subset=['lat', 'lon', 'timestamp', 'vessel_id'])
    
    # Remove duplicate timestamps per vessel
    df = df.sort_values(['vessel_id', 'timestamp'])
    df = df.drop_duplicates(subset=['vessel_id', 'timestamp'], keep='first')
    
    # Remove stationary vessels (speed = 0 for extended periods)
    # Keep vessels with at least some movement
    vessel_max_speed = df.groupby('vessel_id')['speed'].max()
    moving_vessels = vessel_max_speed[vessel_max_speed > 0.5].index
    df = df[df['vessel_id'].isin(moving_vessels)]
    
    removed = original_len - len(df)
    print(f"Data Cleaning:")
    print(f"  Original records: {original_len:,}")
    print(f"  After cleaning: {len(df):,}")
    print(f"  Removed: {removed:,} ({removed/original_len*100:.2f}%)")
    
    return df

# Clean data
ais_clean = clean_ais_data(ais_data)

# Verify data integrity
print(f"\nData Integrity Check:")
print(f"  Lat range: [{ais_clean['lat'].min():.4f}, {ais_clean['lat'].max():.4f}]")
print(f"  Lon range: [{ais_clean['lon'].min():.4f}, {ais_clean['lon'].max():.4f}]")
print(f"  Speed range: [{ais_clean['speed'].min():.2f}, {ais_clean['speed'].max():.2f}] knots")
if 'course' in ais_clean.columns:
    print(f"  Course range: [{ais_clean['course'].min():.2f}, {ais_clean['course'].max():.2f}] degrees")

Data Cleaning:
  Original records: 57,045,766
  After cleaning: 30,358,099
  Removed: 26,687,667 (46.78%)

Data Integrity Check:
  Lat range: [37.4595, 38.0423]
  Lon range: [23.0240, 24.0169]
  Speed range: [0.00, 99.90] knots
  Course range: [0.00, 359.90] degrees


In [5]:
# save in parquet format for faster loading next time
parquet_path = data_root / "ais_cleaned.parquet"
if !parquet_path.exists():
    # parquet_path.unlink()  # Remove existing file
    ais_clean.to_parquet(parquet_path, index=False)
    print(f"\nCleaned data saved to: {parquet_path}")


Cleaned data saved to: ..\dataset\piraeus\ais_cleaned.parquet


## 2. Spoofed Point Generation (Data Augmentation)

Generate synthetic spoofed trajectories using the paper's three classical cases:
1. **Off-course deviation**: Random bearing shifts (±30-90°)
2. **Track deviation**: Gradual position drift with Gaussian noise
3. **CPA violations**: Closest-point-of-approach anomalies

In [6]:
# Spoofing generation utilities
def generate_off_course_spoofing(df, spoof_ratio=0.15):
    """Generate off-course deviation spoofing (random bearing shift)."""
    df_spoofed = df.copy()
    n_spoof = int(len(df) * spoof_ratio)
    spoof_indices = np.random.choice(df.index, size=n_spoof, replace=False)
    
    # Random bearing shift (30-90 degrees)
    bearing_shift = np.random.uniform(30, 90, size=n_spoof)
    direction = np.random.choice([-1, 1], size=n_spoof)
    bearing_shift = bearing_shift * direction
    
    # Apply shift to coordinates (approximate, in degrees)
    # 1 degree ≈ 111 km at equator
    distance_km = np.random.uniform(0.5, 3.0, size=n_spoof)  # 0.5-3 km shift
    
    df_spoofed.loc[spoof_indices, 'lat'] += (distance_km / 111.0) * np.cos(np.radians(bearing_shift))
    df_spoofed.loc[spoof_indices, 'lon'] += (distance_km / (111.0 * np.cos(np.radians(df.loc[spoof_indices, 'lat'])))) * np.sin(np.radians(bearing_shift))
    
    return df_spoofed, spoof_indices


def generate_track_deviation_spoofing(df, spoof_ratio=0.15):
    """Generate track deviation spoofing (gradual drift with Gaussian noise)."""
    df_spoofed = df.copy()
    
    # Select random vessels for spoofing
    vessels = df['vessel_id'].unique()
    n_vessels_spoof = max(1, int(len(vessels) * spoof_ratio))
    spoof_vessels = np.random.choice(vessels, size=n_vessels_spoof, replace=False)
    
    spoof_indices = []
    
    for vessel in spoof_vessels:
        vessel_mask = df['vessel_id'] == vessel
        vessel_data = df[vessel_mask].sort_values('timestamp')
        
        if len(vessel_data) < 10:
            continue
        
        # Apply gradual drift to middle portion of trajectory
        start_idx = len(vessel_data) // 4
        end_idx = 3 * len(vessel_data) // 4
        drift_indices = vessel_data.iloc[start_idx:end_idx].index
        
        # Gaussian noise with increasing magnitude
        drift_length = len(drift_indices)
        lat_drift = np.cumsum(np.random.normal(0, 0.0005, drift_length))
        lon_drift = np.cumsum(np.random.normal(0, 0.0005, drift_length))
        
        df_spoofed.loc[drift_indices, 'lat'] += lat_drift
        df_spoofed.loc[drift_indices, 'lon'] += lon_drift
        
        spoof_indices.extend(drift_indices.tolist())
    
    return df_spoofed, spoof_indices


def generate_cpa_violation_spoofing(df, spoof_ratio=0.15):
    """Generate CPA (Closest Point of Approach) violation spoofing."""
    df_spoofed = df.copy()
    n_spoof = int(len(df) * spoof_ratio)
    spoof_indices = np.random.choice(df.index, size=n_spoof, replace=False)
    
    # Simulate collision-course trajectories (sudden speed/course changes)
    df_spoofed.loc[spoof_indices, 'speed'] *= np.random.uniform(1.5, 3.0, size=n_spoof)
    
    if 'course' in df.columns:
        # Sudden course change
        df_spoofed.loc[spoof_indices, 'course'] += np.random.uniform(-45, 45, size=n_spoof)
        df_spoofed.loc[spoof_indices, 'course'] = df_spoofed.loc[spoof_indices, 'course'] % 360
    
    return df_spoofed, spoof_indices


print("Spoofing generation functions defined")
print("  - Off-course deviation (bearing shift)")
print("  - Track deviation (gradual drift)")
print("  - CPA violation (speed/course anomalies)")

Spoofing generation functions defined
  - Off-course deviation (bearing shift)
  - Track deviation (gradual drift)
  - CPA violation (speed/course anomalies)


In [8]:
# Generate augmented dataset with spoofed points (memory-optimized)
def create_augmented_dataset(df, spoof_ratio=0.3, chunk_size=1_000_000):
    """
    Create augmented dataset with genuine and spoofed AIS messages, optimized for large DataFrames.
    - Uses NumPy arrays and chunked assignment to avoid pandas reindex memory blow-ups.
    - Downcasts numeric columns to float32 to reduce memory usage.
    - Resets the index so NumPy positional arrays align with spoof indices.

    Args:
        df: Clean AIS data
        spoof_ratio: Proportion of data to spoof (0.3 = 30%)
        chunk_size: Number of indices per chunk during assignment

    Returns:
        Augmented dataframe with 'is_spoofed' label
    """
    # Ensure 0-based integer indices for safe NumPy indexing
    df_augmented = df.reset_index(drop=True).copy()
    df_augmented['is_spoofed'] = 0

    # Split spoofing equally among three methods
    ratio_per_method = float(spoof_ratio) / 3.0

    # Generate spoofed data using each method
    df_off_course, indices_off = generate_off_course_spoofing(df_augmented, ratio_per_method)
    df_track_dev, indices_track = generate_track_deviation_spoofing(df_augmented, ratio_per_method)
    df_cpa, indices_cpa = generate_cpa_violation_spoofing(df_augmented, ratio_per_method)

    # Convert to NumPy arrays (downcast to float32 to reduce memory)
    lat = df_augmented['lat'].to_numpy(dtype=np.float32, copy=True)
    lon = df_augmented['lon'].to_numpy(dtype=np.float32, copy=True)
    speed = df_augmented['speed'].to_numpy(dtype=np.float32, copy=True)
    course = df_augmented['course'].to_numpy(dtype=np.float32, copy=True) if 'course' in df_augmented.columns else None

    lat_off = df_off_course['lat'].to_numpy(dtype=np.float32, copy=False)
    lon_off = df_off_course['lon'].to_numpy(dtype=np.float32, copy=False)

    lat_track = df_track_dev['lat'].to_numpy(dtype=np.float32, copy=False)
    lon_track = df_track_dev['lon'].to_numpy(dtype=np.float32, copy=False)

    speed_cpa = df_cpa['speed'].to_numpy(dtype=np.float32, copy=False)
    course_cpa = df_cpa['course'].to_numpy(dtype=np.float32, copy=False) if course is not None and 'course' in df_cpa.columns else None

    # Helper to apply changes in chunks to avoid large temporary arrays
    def apply_in_chunks(idx_array, setter_fn):
        idx_array = np.asarray(idx_array, dtype=np.int64)
        if idx_array.size == 0:
            return
        # Determine chunk count, at least 1
        num_chunks = max(1, int(np.ceil(idx_array.size / chunk_size)))
        for chunk in np.array_split(idx_array, num_chunks):
            if chunk.size == 0:
                continue
            setter_fn(chunk)

    # Apply off-course lat/lon updates
    def set_off_course(chunk):
        lat[chunk] = lat_off[chunk]
        lon[chunk] = lon_off[chunk]
    apply_in_chunks(indices_off, set_off_course)

    # Apply track deviation lat/lon updates
    def set_track_dev(chunk):
        lat[chunk] = lat_track[chunk]
        lon[chunk] = lon_track[chunk]
    apply_in_chunks(indices_track, set_track_dev)

    # Apply CPA speed/course updates
    def set_cpa(chunk):
        speed[chunk] = speed_cpa[chunk]
        if course is not None and course_cpa is not None:
            course[chunk] = course_cpa[chunk]
    apply_in_chunks(indices_cpa, set_cpa)

    # Write back arrays into DataFrame
    df_augmented['lat'] = lat
    df_augmented['lon'] = lon
    df_augmented['speed'] = speed
    if course is not None and 'course' in df_augmented.columns:
        df_augmented['course'] = course

    # Label spoofed points efficiently using a single mask
    mask = np.zeros(len(df_augmented), dtype=np.uint8)
    for idxs in (indices_off, indices_track, indices_cpa):
        if len(idxs) == 0:
            continue
        apply_in_chunks(idxs, lambda ch: mask.__setitem__(ch, 1))
    df_augmented['is_spoofed'] = mask

    # Stats
    total = len(df_augmented)
    genuine = int((df_augmented['is_spoofed'] == 0).sum())
    spoofed = total - genuine

    print("Augmented Dataset Statistics:")
    print(f"  Total records: {total:,}")
    print(f"  Genuine: {genuine:,} ({genuine/total*100:.1f}%)")
    print(f"  Spoofed: {spoofed:,} ({spoofed/total*100:.1f}%)")
    print(f"  - Off-course indices: {len(indices_off):,}")
    print(f"  - Track deviation indices: {len(indices_track):,}")
    print(f"  - CPA violation indices: {len(indices_cpa):,}")

    return df_augmented

# Generate augmented dataset
np.random.seed(42)  # For reproducibility
# If memory issues persist, reduce spoof_ratio (e.g., 0.1) or increase chunk_size
ais_augmented = create_augmented_dataset(ais_clean, spoof_ratio=0.3, chunk_size=1_000_000)

Augmented Dataset Statistics:
  Total records: 30,358,099
  Genuine: 23,413,778 (77.1%)
  Spoofed: 6,944,321 (22.9%)
  - Off-course indices: 3,035,809
  - Track deviation indices: 1,451,125
  - CPA violation indices: 3,035,809


## 3. Feature Engineering & Sequence Creation

Extract features (lat, lon, SOG, COG) and create temporal sequences for Bi-LSTM input.

In [ ]:
# Feature extraction (Algorithm 1: Feature Extraction step)
def extract_features(df):
    """Extract relevant features for Bi-LSTM model."""
    df = df.copy()
    
    # Core features from paper
    features = ['lat', 'lon', 'speed']
    
    if 'course' in df.columns:
        features.append('course')
    
    # Temporal features
    df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['timestamp']).dt.dayofweek
    features.extend(['hour', 'day_of_week'])
    
    # Velocity components (if course available)
    if 'course' in df.columns:
        df['velocity_x'] = df['speed'] * np.cos(np.radians(df['course']))
        df['velocity_y'] = df['speed'] * np.sin(np.radians(df['course']))
        features.extend(['velocity_x', 'velocity_y'])
    
    # Calculate inter-point distances (movement magnitude)
    df = df.sort_values(['vessel_id', 'timestamp'])
    df['lat_diff'] = df.groupby('vessel_id')['lat'].diff()
    df['lon_diff'] = df.groupby('vessel_id')['lon'].diff()
    df['distance'] = np.sqrt(df['lat_diff']**2 + df['lon_diff']**2)
    features.append('distance')
    
    # Fill NaN values from diff operations
    df[features] = df[features].fillna(method='bfill').fillna(method='ffill').fillna(0)
    
    print(f"Feature Extraction Complete:")
    print(f"  Features: {features}")
    print(f"  Feature matrix shape: {df[features].shape}")
    
    return df, features

# Extract features
ais_featured, feature_cols = extract_features(ais_augmented)

# Display sample
print(f"\nSample Features:")
print(ais_featured[feature_cols + ['is_spoofed']].head(10))

In [ ]:
# Feature normalization (Algorithm 1: Feature Scaling)
def normalize_features(df, feature_cols, fit_scaler=True, scaler=None):
    """Normalize features using StandardScaler."""
    if fit_scaler:
        scaler = StandardScaler()
        df[feature_cols] = scaler.fit_transform(df[feature_cols])
        print(f"Features normalized (fitted new scaler)")
    else:
        df[feature_cols] = scaler.transform(df[feature_cols])
        print(f"Features normalized (using existing scaler)")
    
    return df, scaler

# Normalize features
ais_normalized, feature_scaler = normalize_features(
    ais_featured, 
    feature_cols, 
    fit_scaler=True
)

print(f"\nNormalized Feature Statistics:")
print(ais_normalized[feature_cols].describe())

In [ ]:
# Create sequences for LSTM (temporal windows)
def create_sequences(df, feature_cols, sequence_length=128, stride=32):
    """
    Create temporal sequences for Bi-LSTM input.
    
    Args:
        df: Dataframe with features
        feature_cols: List of feature column names
        sequence_length: Number of timesteps per sequence
        stride: Step size for sliding window
    
    Returns:
        X: Feature sequences (n_samples, sequence_length, n_features)
        y: Labels (n_samples,)
    """
    X_sequences = []
    y_labels = []
    
    # Group by vessel
    for vessel_id, vessel_data in df.groupby('vessel_id'):
        vessel_data = vessel_data.sort_values('timestamp')
        
        # Extract features and labels
        features = vessel_data[feature_cols].values
        labels = vessel_data['is_spoofed'].values
        
        # Skip if insufficient data
        if len(features) < sequence_length:
            continue
        
        # Create sliding windows
        for i in range(0, len(features) - sequence_length + 1, stride):
            seq_features = features[i:i+sequence_length]
            seq_label = labels[i:i+sequence_length]
            
            # Label sequence as spoofed if ANY point is spoofed
            # (conservative approach for safety)
            is_spoofed = int(seq_label.sum() > 0)
            
            X_sequences.append(seq_features)
            y_labels.append(is_spoofed)
    
    X = np.array(X_sequences)
    y = np.array(y_labels)
    
    print(f"Sequence Creation:")
    print(f"  Sequence length: {sequence_length} timesteps")
    print(f"  Stride: {stride}")
    print(f"  Total sequences: {len(X):,}")
    print(f"  Genuine sequences: {(y == 0).sum():,} ({(y == 0).sum()/len(y)*100:.1f}%)")
    print(f"  Spoofed sequences: {(y == 1).sum():,} ({(y == 1).sum()/len(y)*100:.1f}%)")
    print(f"  Shape: X={X.shape}, y={y.shape}")
    
    return X, y

# Create sequences
SEQUENCE_LENGTH = 128
STRIDE = 32

X_sequences, y_labels = create_sequences(
    ais_normalized, 
    feature_cols, 
    sequence_length=SEQUENCE_LENGTH,
    stride=STRIDE
)

In [ ]:
# Train/test split (Algorithm 1: train_test_split)
# Split by vessel to avoid leakage
def vessel_aware_split(X, y, df, test_size=0.2, val_size=0.1):
    """
    Split data by vessel to prevent data leakage.
    """
    # Get unique vessels
    vessels = df['vessel_id'].unique()
    n_vessels = len(vessels)
    
    # Shuffle vessels
    np.random.shuffle(vessels)
    
    # Calculate split indices
    test_idx = int(n_vessels * (1 - test_size))
    val_idx = int(test_idx * (1 - val_size))
    
    train_vessels = vessels[:val_idx]
    val_vessels = vessels[val_idx:test_idx]
    test_vessels = vessels[test_idx:]
    
    # Create vessel-to-sequences mapping
    vessel_to_seq = {}
    for idx, (vessel_id, _) in enumerate(df.groupby('vessel_id')):
        vessel_to_seq[vessel_id] = []
    
    # Map sequences to vessels (approximate based on order)
    # This is simplified; in production, track vessel_id through sequence creation
    sequences_per_vessel = len(X) // n_vessels
    
    train_mask = []
    val_mask = []
    test_mask = []
    
    for i in range(len(X)):
        # Simple assignment (can be improved with proper tracking)
        vessel_idx = min(i // max(1, sequences_per_vessel), n_vessels - 1)
        vessel = vessels[vessel_idx]
        
        if vessel in train_vessels:
            train_mask.append(i)
        elif vessel in val_vessels:
            val_mask.append(i)
        else:
            test_mask.append(i)
    
    X_train = X[train_mask]
    y_train = y[train_mask]
    X_val = X[val_mask]
    y_val = y[val_mask]
    X_test = X[test_mask]
    y_test = y[test_mask]
    
    print(f"Data Split (vessel-aware):")
    print(f"   Train: {len(X_train):,} sequences ({len(X_train)/len(X)*100:.1f}%)")
    print(f"   Val:   {len(X_val):,} sequences ({len(X_val)/len(X)*100:.1f}%)")
    print(f"   Test:  {len(X_test):,} sequences ({len(X_test)/len(X)*100:.1f}%)")
    print(f"\n   Train - Genuine: {(y_train==0).sum()}, Spoofed: {(y_train==1).sum()}")
    print(f"   Val   - Genuine: {(y_val==0).sum()}, Spoofed: {(y_val==1).sum()}")
    print(f"   Test  - Genuine: {(y_test==0).sum()}, Spoofed: {(y_test==1).sum()}")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Alternative: Simple random split (faster, but may have leakage)
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_labels, test_size=0.2, random_state=42, stratify=y_labels
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.125, random_state=42, stratify=y_train  # 0.125 * 0.8 = 0.1 of total
)

print(f"Data Split (random stratified):")
print(f"  Train: {len(X_train):,} sequences")
print(f"  Val:   {len(X_val):,} sequences")
print(f"  Test:  {len(X_test):,} sequences")
print(f"\n  Train - Genuine: {(y_train==0).sum()}, Spoofed: {(y_train==1).sum()}")
print(f"  Val   - Genuine: {(y_val==0).sum()}, Spoofed: {(y_val==1).sum()}")
print(f"  Test  - Genuine: {(y_test==0).sum()}, Spoofed: {(y_test==1).sum()}")

## 4. Bi-LSTM Model Architecture

PyTorch implementation of Algorithm 2 from the paper:
- Input Layer
- Bidirectional LSTM(62 units, return_sequences=True)
- Bidirectional LSTM(30 units)
- Dense(1, sigmoid)
- Binary crossentropy loss, Adam optimizer


In [ ]:
# Build Bi-LSTM model (Algorithm 2 from paper) - PyTorch
class BiLSTMModel(nn.Module):
    """
    PyTorch Bi-LSTM model for AIS spoofing detection.
    
    Architecture:
    - Bidirectional LSTM(62, return_sequences=True)
    - Bidirectional LSTM(30)
    - Dense(1, sigmoid)
    
    Args:
        input_size: Number of features
        lstm_units_1: First Bi-LSTM layer units (default: 62 from paper)
        lstm_units_2: Second Bi-LSTM layer units (default: 30 from paper)
    """
    def __init__(self, input_size, lstm_units_1=62, lstm_units_2=30):
        super(BiLSTMModel, self).__init__()
        
        # First Bidirectional LSTM (bidirectional doubles the output size)
        self.bilstm1 = nn.LSTM(
            input_size=input_size,
            hidden_size=lstm_units_1,
            batch_first=True,
            bidirectional=True,
            dropout=0.2
        )
        
        # Second Bidirectional LSTM
        self.bilstm2 = nn.LSTM(
            input_size=lstm_units_1 * 2,  # Bidirectional output is doubled
            hidden_size=lstm_units_2,
            batch_first=True,
            bidirectional=True,
            dropout=0.2
        )
        
        # Output layer
        self.fc = nn.Linear(lstm_units_2 * 2, 1)  # Bidirectional output is doubled
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        """Forward pass through the model."""
        # First Bi-LSTM layer
        lstm1_out, _ = self.bilstm1(x)
        
        # Second Bi-LSTM layer
        lstm2_out, _ = self.bilstm2(lstm1_out)
        
        # Take the last output from the second LSTM
        last_hidden = lstm2_out[:, -1, :]
        
        # Dense layer + Sigmoid
        output = self.fc(last_hidden)
        output = self.sigmoid(output)
        
        return output


def build_bilstm_model(input_size, lstm_units_1=62, lstm_units_2=30):
    """Create and return Bi-LSTM model."""
    model = BiLSTMModel(input_size, lstm_units_1, lstm_units_2)
    return model.to(device)


# Build model
print("Building Bi-LSTM Model (PyTorch)...")
print(f"  Input size: {X_train.shape[2]} features")

model = build_bilstm_model(
    input_size=X_train.shape[2],
    lstm_units_1=62,  # From paper
    lstm_units_2=30   # From paper
)

# Display model architecture
print("\nModel Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## 5. Model Training

Train with hyperparameters from Table 1 in paper:
- LSTM units: 62, 30
- Batch size: 30
- Learning rate: 0.004
- Epochs: 50
- Early stopping with patience

In [ ]:
# Training configuration (from paper Table 1)
BATCH_SIZE = 30  # From paper
EPOCHS = 50  # From paper
LEARNING_RATE = 0.004  # From paper
PATIENCE = 10  # Early stopping patience

# Create data loaders
train_dataset = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float().reshape(-1, 1)
)
val_dataset = TensorDataset(
    torch.from_numpy(X_val).float(),
    torch.from_numpy(y_val).float().reshape(-1, 1)
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"Training Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Early stopping patience: {PATIENCE}")
print(f"  Device: {device}")
print(f"\nStarting training...\n")

In [ ]:
# Train model (Algorithm 2: Custom PyTorch training loop)
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    for batch_x, batch_y in loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * batch_x.size(0)
        all_preds.extend(outputs.detach().cpu().numpy())
        all_labels.extend(batch_y.detach().cpu().numpy())
    
    return total_loss / len(loader.dataset), np.array(all_preds), np.array(all_labels)


def validate(model, loader, criterion, device):
    """Validate the model."""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            total_loss += loss.item() * batch_x.size(0)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    return total_loss / len(loader.dataset), np.array(all_preds), np.array(all_labels)


# Training loop with early stopping
history = {
    'train_loss': [],
    'val_loss': [],
    'train_accuracy': [],
    'val_accuracy': [],
    'train_precision': [],
    'val_precision': [],
    'train_recall': [],
    'val_recall': []
}

best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(EPOCHS):
    # Train
    train_loss, train_preds, train_labels = train_epoch(model, train_loader, criterion, optimizer, device)
    train_preds_binary = (train_preds > 0.6).astype(int)
    
    # Validate
    val_loss, val_preds, val_labels = validate(model, val_loader, criterion, device)
    val_preds_binary = (val_preds > 0.6).astype(int)
    
    # Calculate metrics
    train_acc = (train_preds_binary == train_labels).mean()
    val_acc = (val_preds_binary == val_labels).mean()
    
    train_prec = precision_score(train_labels, train_preds_binary, zero_division=0)
    val_prec = precision_score(val_labels, val_preds_binary, zero_division=0)
    
    train_rec = recall_score(train_labels, train_preds_binary, zero_division=0)
    val_rec = recall_score(val_labels, val_preds_binary, zero_division=0)
    
    # Store history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_accuracy'].append(train_acc)
    history['val_accuracy'].append(val_acc)
    history['train_precision'].append(train_prec)
    history['val_precision'].append(val_prec)
    history['train_recall'].append(train_rec)
    history['val_recall'].append(val_rec)
    
    # Print progress
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"\nEarly stopping at epoch {epoch+1}")
            model.load_state_dict(best_model_state)
            break

print(f"\nTraining Complete!")
print(f"  Best epoch: {np.argmin(history['val_loss']) + 1}")
print(f"  Best val_loss: {min(history['val_loss']):.4f}")
print(f"  Best val_accuracy: {max(history['val_accuracy']):.4f}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Model Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Model Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Precision & Recall
axes[2].plot(history['train_precision'], label='Train Precision', linewidth=2)
axes[2].plot(history['val_precision'], label='Val Precision', linewidth=2, linestyle='--')
axes[2].plot(history['train_recall'], label='Train Recall', linewidth=2)
axes[2].plot(history['val_recall'], label='Val Recall', linewidth=2, linestyle='--')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Score')
axes[2].set_title('Precision & Recall')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_root / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Training plots saved to: {output_root / 'training_history.png'}")

## 6. Model Evaluation

Evaluate on test set and compare with paper's reported metrics (Table 2).

In [ ]:
# Predict on test set (Algorithm 3: Anomaly Score Calculation)
print("Generating predictions on test set...\n")

# Convert test data to tensors
X_test_tensor = torch.from_numpy(X_test).float().to(device)
y_test_tensor = torch.from_numpy(y_test).float()

# Get predictions
model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor).cpu().numpy()

y_pred_probs = y_pred_probs.flatten()

# Apply threshold τ = 0.6 (from paper section 3.3.2)
THRESHOLD = 0.6  # Optimal threshold from paper
y_pred = (y_pred_probs > THRESHOLD).astype(int)

# Calculate metrics
test_loss = criterion(torch.from_numpy(y_pred_probs).float().to(device), 
                      torch.from_numpy(y_test).float().reshape(-1, 1).to(device)).item()
test_acc = (y_pred == y_test).mean()

# Additional metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Detection rate (same as recall)
detection_rate = recall

print(f"Test Set Evaluation (threshold τ = {THRESHOLD}):")
print(f"  Accuracy:       {test_acc:.4f}")
print(f"  Precision:      {precision:.4f}")
print(f"  Recall:         {recall:.4f}")
print(f"  F1-Score:       {f1:.4f}")
print(f"  Detection Rate: {detection_rate:.4f}")

# Compare with paper's Bi-LSTM results (Table 2)
paper_results = {
    "Precision": 0.94,
    "Recall": 0.92,
    "F1-Score": 0.93,
    "Detection Rate": 0.91
}

print(f"\nPaper's Reported Bi-LSTM Results:")
for metric, value in paper_results.items():
    print(f"  {metric}: {value:.2f}")

print(f"\nPerformance Comparison:")
print(f"  Precision:      {'✓' if precision >= 0.85 else '✗'} (Ours: {precision:.2f}, Paper: {paper_results['Precision']:.2f})")
print(f"  Recall:         {'✓' if recall >= 0.85 else '✗'} (Ours: {recall:.2f}, Paper: {paper_results['Recall']:.2f})")
print(f"  F1-Score:       {'✓' if f1 >= 0.85 else '✗'} (Ours: {f1:.2f}, Paper: {paper_results['F1-Score']:.2f})")
print(f"  Detection Rate: {'✓' if detection_rate >= 0.85 else '✗'} (Ours: {detection_rate:.2f}, Paper: {paper_results['Detection Rate']:.2f})")

In [ ]:
# Confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, cmap='Blues')

# Labels
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Genuine', 'Spoofed'])
ax.set_yticklabels(['Genuine', 'Spoofed'])

# Annotations
for i in range(2):
    for j in range(2):
        text = ax.text(j, i, cm[i, j], ha="center", va="center", 
                      color="white" if cm[i, j] > cm.max() / 2 else "black",
                      fontsize=20, fontweight='bold')

ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)
ax.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.savefig(output_root / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Classification report
print(f"\nDetailed Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Genuine', 'Spoofed']))

print(f"\nConfusion matrix saved to: {output_root / 'confusion_matrix.png'}")

## 7. Spoofing Detection on Real Incidents

Apply Algorithm 3 to detect spoofed points in actual incident trajectories.

In [ ]:
# Detect spoofed points in real AIS data (Algorithm 3 implementation)
def detect_spoofing_in_data(model, df, feature_cols, scaler, sequence_length=128, threshold=0.6):
    """
    Apply trained model to detect spoofed AIS messages in real data.
    
    Returns:
        DataFrame with anomaly scores and predictions
    """
    # Prepare features
    df_featured, _ = extract_features(df.copy())
    df_normalized, _ = normalize_features(df_featured, feature_cols, fit_scaler=False, scaler=scaler)
    
    # Create sequences per vessel
    results = []
    
    for vessel_id, vessel_data in df_normalized.groupby('vessel_id'):
        vessel_data = vessel_data.sort_values('timestamp')
        features = vessel_data[feature_cols].values
        
        if len(features) < sequence_length:
            # Skip vessels with insufficient data
            continue
        
        # Create overlapping windows
        for i in range(len(features) - sequence_length + 1):
            seq = features[i:i+sequence_length]
            seq = seq.reshape(1, sequence_length, len(feature_cols))
            
            # Predict anomaly score
            score = model.predict(seq, verbose=0)[0][0]
            
            # Store result for middle point of sequence
            mid_idx = i + sequence_length // 2
            if mid_idx < len(vessel_data):
                row = vessel_data.iloc[mid_idx].copy()
                row['anomaly_score'] = score
                row['is_spoofed'] = 1 if score > threshold else 0
                results.append(row)
    
    if not results:
        return None
    
    results_df = pd.DataFrame(results)
    
    print(f"Spoofing Detection Results:\")\n    print(f\"  Total points analyzed: {len(results_df):,}\")\n    print(f\"  Detected spoofed: {(results_df['is_spoofed']==1).sum():,} ({(results_df['is_spoofed']==1).sum()/len(results_df)*100:.1f}%)\")\n    print(f\"  Mean anomaly score: {results_df['anomaly_score'].mean():.3f}\")\n    print(f\"  Max anomaly score: {results_df['anomaly_score'].max():.3f}\")\n    \n    return results_df\n\n# Example: Load and test on a specific month\nprint(\"Testing spoofing detection on sample data...\")\ntry:\n    # Load a test month (e.g., Sept 2017 - Agia Zoni II incident)\n    test_data = load_month(2017, 9, data_root)\n    print(f\"Loaded test data: {len(test_data):,} records\")\n    \n    # Detect spoofing\n    incident_results = detect_spoofing_in_data(\n        model, test_data, feature_cols, feature_scaler, \n        sequence_length=SEQUENCE_LENGTH, threshold=THRESHOLD\n    )\n    \n    if incident_results is not None:\n        print(f\"\\nHigh-risk detections (score > 0.8):\")\n        high_risk = incident_results[incident_results['anomaly_score'] > 0.8]\n        print(f\"  Count: {len(high_risk)}\")\n        if len(high_risk) > 0:\n            print(high_risk[['timestamp', 'vessel_id', 'lat', 'lon', 'speed', 'anomaly_score']].head(10))\nexcept Exception as e:\n    print(f\"Could not run incident test: {e}\")\n    incident_results = None

In [ ]:
# Detect spoofed points in a specific incident (Algorithm 3 implementation)
def detect_spoofing_in_incident(incident_name, model, feature_scaler, feature_cols, 
                                 threshold=0.6, sequence_length=128):
    """
    Apply spoofing detection to a specific incident.
    
    Returns dataframe with anomaly scores for each AIS message.
    """
    # Load incident data
    inc = next((i for i in incidents if i["name"] == incident_name), None)
    if inc is None:
        print(f"❌ Incident '{incident_name}' not found")
        return None
    
    slice_path = data_root / inc["slug"] / "slice.parquet"
    if not slice_path.exists():
        print(f"❌ Data not found: {slice_path}")
        return None
    
    # Load and preprocess
    df = pd.read_parquet(slice_path)
    df = clean_ais_data(df)
    
    # Extract and normalize features
    df, _ = extract_features(df)
    df, _ = normalize_features(df, feature_cols, fit_scaler=False, scaler=feature_scaler)
    
    # Create sequences per vessel
    anomaly_scores = []
    
    for vessel_id, vessel_data in df.groupby('vessel_id'):
        vessel_data = vessel_data.sort_values('timestamp')
        features = vessel_data[feature_cols].values
        
        if len(features) < sequence_length:
            # For short trajectories, use available length
            seq = features
            if len(seq) < 10:
                continue
            # Pad if needed
            if len(seq) < sequence_length:
                pad_length = sequence_length - len(seq)
                seq = np.vstack([seq, np.zeros((pad_length, seq.shape[1]))])
        else:
            # Use sliding window
            seq = features[:sequence_length]
        
        # Predict
        seq_reshaped = seq.reshape(1, sequence_length, -1)
        score = model.predict(seq_reshaped, verbose=0)[0][0]
        
        # Store scores for all points in this vessel
        for idx in vessel_data.index:
            anomaly_scores.append({
                'index': idx,
                'vessel_id': vessel_id,
                'anomaly_score': score,
                'is_spoofed': int(score > threshold)
            })
    
    # Merge with original data
    scores_df = pd.DataFrame(anomaly_scores)
    df_with_scores = df.merge(scores_df, left_index=True, right_on='index', how='left')
    
    return df_with_scores

# Test on Agia Zoni II incident
print(" Detecting spoofing in Agia Zoni II incident...\n")
incident_results = detect_spoofing_in_incident(
    "Agia Zoni II", 
    model, 
    feature_scaler, 
    feature_cols,
    threshold=THRESHOLD
)

if incident_results is not None:
    print(f"✓ Detection complete for Agia Zoni II")
    print(f"   Total AIS messages: {len(incident_results)}")
    print(f"   Detected spoofed: {incident_results['is_spoofed'].sum()}")
    print(f"   Detection rate: {incident_results['is_spoofed'].sum() / len(incident_results) * 100:.2f}%")
    print(f"\n   Anomaly score statistics:")
    print(f"   Mean: {incident_results['anomaly_score'].mean():.4f}")
    print(f"   Std:  {incident_results['anomaly_score'].std():.4f}")
    print(f"   Max:  {incident_results['anomaly_score'].max():.4f}")
    print(f"   Min:  {incident_results['anomaly_score'].min():.4f}")

In [ ]:
# Visualize detected spoofing on map
if incident_results is not None:
    inc = next((i for i in incidents if i["name"] == "Agia Zoni II"), None)
    lat0, lon0 = inc["approx_lat"], inc["approx_lon"]
    
    # Create map
    m = folium.Map(
        location=[lat0, lon0],
        zoom_start=12,
        tiles="CartoDB positron"
    )
    
    # Feature groups
    fg_genuine = folium.FeatureGroup(name="Genuine AIS", show=True)
    fg_spoofed = folium.FeatureGroup(name="Detected Spoofed", show=True)
    fg_incident = folium.FeatureGroup(name="Incident Location", show=True)
    
    # Plot trajectories by vessel and spoofing status
    for vessel_id, vessel_data in incident_results.groupby('vessel_id_x'):
        vessel_data = vessel_data.sort_values('timestamp')
        
        # Split into genuine and spoofed segments
        genuine = vessel_data[vessel_data['is_spoofed'] == 0]
        spoofed = vessel_data[vessel_data['is_spoofed'] == 1]
        
        # Genuine trajectory (blue)
        if len(genuine) > 1:
            coords_genuine = genuine[['lat', 'lon']].values.tolist()
            folium.PolyLine(
                coords_genuine, 
                color='blue', 
                weight=2, 
                opacity=0.7,
                tooltip=f"Vessel {vessel_id} (Genuine)"
            ).add_to(fg_genuine)
        
        # Spoofed points (red)
        if len(spoofed) > 0:
            for idx, row in spoofed.iterrows():
                folium.CircleMarker(
                    location=[row['lat'], row['lon']],
                    radius=6,
                    color='red',
                    fill=True,
                    fillColor='red',
                    fillOpacity=0.8,
                    popup=f"Spoofed<br>Score: {row['anomaly_score']:.3f}<br>Vessel: {vessel_id}",
                    tooltip="Detected Spoofing"
                ).add_to(fg_spoofed)
    
    # Mark incident location
    folium.Marker(
        location=[lat0, lon0],
        popup=f"<b>Agia Zoni II Incident</b><br>2017-09-10",
        icon=folium.Icon(color="red", icon="exclamation", prefix="fa"),
        tooltip="Incident Location"
    ).add_to(fg_incident)
    
    # Add layers
    fg_genuine.add_to(m)
    fg_spoofed.add_to(m)
    fg_incident.add_to(m)
    folium.LayerControl().add_to(m)
    
    print(f"  Spoofing detection map created")
    display(m)

## 8. Model Export & Summary

Save trained model, scaler, and configuration for deployment.

In [ ]:
# Save model artifacts
import pickle

# Save PyTorch model
model_path = output_root / 'best_bilstm_model.pt'
torch.save(model.state_dict(), model_path)

# Save scaler
scaler_path = output_root / 'feature_scaler.pkl'
with open(scaler_path, 'wb') as f:
    pickle.dump(feature_scaler, f)

# Save configuration
config = {
    'feature_cols': feature_cols,
    'sequence_length': SEQUENCE_LENGTH,
    'threshold': THRESHOLD,
    'lstm_units_1': 62,
    'lstm_units_2': 30,
    'learning_rate': LEARNING_RATE,
    'batch_size': BATCH_SIZE,
    'input_size': X_train.shape[2],
    'test_metrics': {
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'detection_rate': float(detection_rate),
        'accuracy': float(test_acc)
    }
}

config_path = output_root / 'model_config.pkl'
with open(config_path, 'wb') as f:
    pickle.dump(config, f)

print(f"Model Artifacts Saved:")
print(f"  Model: {model_path}")
print(f"  Scaler: {scaler_path}")
print(f"  Config: {config_path}")

print(f"\n" + "="*60)
print(f"Bi-LSTM SPOOFING DETECTION - PYTORCH IMPLEMENTATION COMPLETE")
print(f"="*60)
print(f"\nFinal Test Set Performance:")
print(f"  Precision:      {precision:.4f}")
print(f"  Recall:         {recall:.4f}")
print(f"  F1-Score:       {f1:.4f}")
print(f"  Detection Rate: {detection_rate:.4f}")
print(f"  Accuracy:       {test_acc:.4f}")

print(f"\nComparison with Paper (Bi-LSTM):")
print(f"  Precision:      Paper: 0.94, Ours: {precision:.2f}")
print(f"  Recall:         Paper: 0.92, Ours: {recall:.2f}")
print(f"  F1-Score:       Paper: 0.93, Ours: {f1:.2f}")
print(f"  Detection Rate: Paper: 0.91, Ours: {detection_rate:.2f}")

print(f"\nNext Steps:")
print(f"  1. Apply to all 5 maritime incidents")
print(f"  2. Fine-tune threshold τ for production use")
print(f"  3. Integrate with TimeFM for trajectory forecasting")
print(f"  4. Deploy for real-time AIS stream processing")
print(f"="*60)